<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado el 2018-1, 2018-2 por Equipo docente IIC2233</font>
</p>

## Colas (_queues_)

Una cola es una estructura de datos secuencial que mantiene objetos ordenados de acuerdo a su orden de llegada. Funciona igual a una cola para pagar las cuentas en el mundo real, pues cada vez que llega una persona ésta se coloca al final, y la persona que está al principio es atendida y sale de la fila. Es por esto que una cola es una estructura de tipo _**First-in, First-out**_ (FIFO).

![](img/queues.png)

Una cola tiene dos operaciones principales:
- _**Enqueue**_: Agrega un elemento al final de la cola.
- _**Dequeue**_: Saca el elemento que está al inicio de la cola. Esto siempre sacará el elemento que lleva más tiempo en la cola.

Además, tiene una operación _**peek**_ que permite ver el primer elemento de la cola sin sacarlo, y es posible revisar la cantidad de elementos en la cola o si ésta se encuentra vacía.

Si intentamos implementar una cola usando una lista de Python, encontraremos que implementar el _**enqueue**_ es directo pues basta con realizar un `append` a la lista. Sin embargo, implementar _**dequeue**_ mediante `pop(0)` **no es eficiente**. En una lista con $N$ elementos, al eliminar el elemento de la posición $0$, la lista queda de largo $N-1$, y el espacio que ocupaba ese elemento queda ahora vacío en la memoria del computador. La lista desplaza todos los otros elementos una posición a la izquierda: el de la posición $1$ pasa a la posición $0$, el de la posición $2$ pasa a la posición $1$, $\dots,$ hasta que el de la posición $N$ pasa a la posición $N-1$. Por lo tanto, al ejecutar `pop(0)` se realizan $N$ operaciones: eliminar el elemento, y mover $N-1$ elementos a la izquierda. Decimos que esto es mucho más _costoso_ que agregar un elemento a la cola (con `append`), donde solo se realiza solo una operación.


### Implementación en Python

El módulo `collections` no provee exactamente la estructura _queue_, sino que una versión con más operaciones llamada  _deque_ (por _double ended queue_). Por ahora, nos limitaremos a indicar cómo implementar las operaciones de las colas con la clase `deque`, y en la siguiente sección profundizaremos en esta estructura.


| Operación                 | Código Python           | Descripción                                           |
|---------------------------|-------------------------|-------------------------------------------------------|
| Crear cola                | `cola = deque()`        | Crea una cola vacía                                   |
| Crear cola                | `cola = deque(lista)`   | Crea una cola a partir de los elementos de una lista  |
| _Enqueue_                 | `cola.append(elemento)` | Agrega un elemento al final de la cola                |
| _Dequeue_                 | `cola.popleft()`        | Retorna y extrae el elemento del principio de la cola |
| _Peek_                    | `cola[0]`               | Retorna el primer elemento de la cola sin extraerlo   |
| _length_                  | `len(cola)`             | Retorna la cantidad de elementos en la cola           |
|*is_empty*                 | `len(cola) == 0`        | Retorna true si la cola está vacía                    |


#### Ejemplo básico de funcionamiento

In [1]:
# Importamos deque
from collections import deque

# Cola vacía
cola = deque()

# Agregamos elementos a la cola (enqueue)
cola.append(1)
cola.append(2)
cola.append(3)

cola

deque([1, 2, 3])

Ahora, mostramos como hacer _dequeue_ con `popleft`

In [2]:
elemento = cola.popleft()

print(f"Hicimos dequeue de {elemento}.")
print(f"La cola quedó: {cola}")

Hicimos dequeue de 1.
La cola quedó: deque([2, 3])


Podemos ver el primer elemento de la cola sin extraerlo _(peek)_.

In [3]:
print(f"Primer elemento de la cola: {cola[0]}")

Primer elemento de la cola: 2


Por último, podemos ver cuántos elementos hay y si la cola está vacía.

In [4]:
# len
print(f"La cola tiene {len(cola)} elementos.")

# Función para revisar si la cola está vacía
def is_empty(s):
    return len(s) == 0

print(f"¿La cola está vacía? {is_empty(cola)}")

La cola tiene 2 elementos.
¿La cola está vacía? False


In [5]:
cola.popleft()
print(f"Cola: {cola}")
print(f"¿La cola está vacía? {is_empty(cola)}")

Cola: deque([3])
¿La cola está vacía? False


In [6]:
cola.popleft()
print(f"Cola: {cola}")
print(f"¿La cola está vacía? {is_empty(cola)}")

Cola: deque([])
¿La cola está vacía? True


### Ejemplo de uso

Casos de uso típicos de esta estructura de datos son las colas de impresión, atención de clientes, atención en cajeros automáticos, etc. En el ejemplo a continuación veremos un modelo simple para atención en una línea de revisión técnica.

In [7]:
from collections import deque
from random import choice, randrange


class Auto:
    """Esta clase modela los autos que llegan a la revision."""
    
    # Un staticmethod es un método propio de la clase, que no se aplica a una instancia particular
    # Su primer argumento NO es una instancia de la clase (al que solemos llamar self)
    @staticmethod
    def tiempo_revision(tipo):
        if tipo == 'auto':
            return 10
        elif tipo == 'moto':
            return 25
        return 30
    
    def __init__(self):
        self.tipo_vehiculo = choice(['moto', 'camioneta', 'auto'])
        self.tiempo_revision = self.tiempo_revision(self.tipo_vehiculo)
        
    def __str__(self):
        return self.tipo_vehiculo


class Taller:
    """
    Esta clase modela la linea de revision en el taller.
    """
    
    def __init__(self):
        self.auto_actual = None
        self.tiempo_actual = 0

    def ocupado(self):
        """
        Verifica si el taller está ocupado. 
        Retorna False cuando está vacío.
        """
        return self.auto_actual != None

    def ingresar_auto(self, auto):
        self.auto_actual = auto
        self.tiempo_actual = 0
        print("Atendiendo: {}".format(self.auto_actual))
        
    def tick(self):
        """
        Realiza el incremento del contador de tiempo 
        en la simulación.
        """
        if self.auto_actual != None:
            self.tiempo_actual += 1
            if self.auto_actual.tiempo_revision == self.tiempo_actual:
                self.auto_actual = None
                self.tiempo_actual = 0

        
def llega_nuevo_auto():
    """
    La llegada de los vehículos a la línea de 
    revisión está modelada como un proceso aleatorio.
    Por cada intervalo de tiempo, hay un 1% de 
    probabilidad de que llegue un auto
    """
    return 0 == randrange(0, 100)


def revision_tecnica():
    """Esta función maneja el proceso de revisión."""
    
    planta_revision = Taller()  # Crea una planta de revisión
    cola_revision = deque()  # Cola de revision vacia
    tiempo_espera = []  # Lista con los tiempos de espera

    
    # Simulación por 500 intervalos de tiempo
    for _ in range(500):
        
        if llega_nuevo_auto():
            auto = Auto()
            cola_revision.append(auto)
        
        if (not planta_revision.ocupado()) and (len(cola_revision) > 0):
            # Extrae el próximo auto en la cola de atención y 
            # lo pasa a la planta de revisión.
            proximo_auto = cola_revision.popleft()
            tiempo_espera.append(proximo_auto.tiempo_revision)
            planta_revision.ingresar_auto(proximo_auto)
    
        planta_revision.tick()  # Incrementa el tiempo que el auto ha sido atendido

    tiempo_promedio = sum(tiempo_espera) / len(tiempo_espera)
    print(f"""Tiempo promedio de atención: {tiempo_promedio:6.2f} minutos """
          f"""y {len(cola_revision)} vehiculos aún esperando.""")


revision_tecnica()

Atendiendo: moto
Atendiendo: camioneta
Atendiendo: camioneta
Tiempo promedio de atención:  28.33 minutos y 0 vehiculos aún esperando.


## Colas de doble extremo (_Deque_)


Un _deque_ (_double-ended queue_, lo pronunciamos "dec") es una estructura secuencial en la que es posible **agregar y sacar elementos desde ambos extremos en forma eficiente**, con un _costo constante por operación_. Esto quiere decir que, independientemente del largo de un _deque_, si éste tiene $N$ elementos, para agregar y sacar elementos siempre ejecutará _la misma_ cantidad de operaciones. Esto es mucho mejor que si utilizamos una _lista_, en que la cantidad de operaciones depende de la cantidad de elementos en la lista. En Python, esta estructura es provista por la clase `deque` del módulo `collections`. Las principales operaciones que soporta son:

| Operación      | Código Python                | Descripción                                                      |
|----------------|------------------------------|------------------------------------------------------------------|
| Crear _deque_  | `deque()`                    | Crea un _deque_ vacío                                            |
| Crear _deque_  | `deque(lista)`               | Crea un _deque_ a partir de los elementos de una lista           |
| _Add first_    | `deque.appendleft(elemento)` | Agrega un elemento al inicio del _deque_                         |
| _Add last_     | `deque.append(elemento)`     | Agrega un elemento al final del _deque_                          |
| _Delete first_ | `deque.popleft()`            | Retorna y extrae el primer elemento del _deque_                  |
| _Delete last_  | `deque.pop()`                | Retorna y extrae el último elemento del _deque_                  |
| _First_        | `deque[0]`                   | Retorna sin extraer el primer elemento del _deque_               |
| _Last_         | `deque[-1]`                  | Retorna sin extraer el último elemento del _deque_               |
| _length_       | `len(deque)`                 | Retorna el número de elementos en el _deque_                     |
| _Is empty_     | `len(deque) == 0`            | Retorna true si el _deque_ está vacío                            |
| _Clear_        | `deque.clear()`              | Limpia el _deque_                                                |
| _Remove_       | `deque.remove(elemento)`     | Saca el primer elemento del _deque_ que sea igual a `elemento`   |
| _Count_        | `deque.count(elemento)`      | Cuenta el número de elementos iguales a `elemento` en el _deque_ |


El _deque_ soporta acceso de lectura y escritura en el elemento de la posición `i`, con la sentencia `deque[i]`. Sin embargo, esta operación _**no es eficiente**_ como en el caso de las listas. En un _deque_, para llegar a la posición `i` el computador inicia en la posición `0` y se va moviendo hasta encontrar la posición `i` para poder leerlo (es decir, requiere recorrer todos los elementos anteriores a `i` para llegar a `i`). 

#### Comparando `list` y `deque`

En el siguiente código vamos a comparar el tiempo que demoran algunas operaciones en un `deque` y en un `list`. Vamos a crear un `deque` y un `list` de 10 millones de enteros cada uno, y luego vamos a comparar (1) el tiempo que toma encontrar el elemento que está en la mitad de cada uno, y (2) el tiempo que toma sacar 1000 elementos del inicio.

In [8]:
from collections import deque
from time import time

ELEMENTS = 10000000

# Creamos un deque y una lista con 10.000.000 de enteros
number_deque = deque(range(ELEMENTS))
number_list = list(range(ELEMENTS))

# Vemos el time actual
start_time = time()
# Buscamos el elemento del medio
number_deque[ELEMENTS // 2]
finish_time = time()
deque_time = finish_time - start_time
# Imprimimos el tiempo transcurrido
print(f"""Buscar el elemento {ELEMENTS // 2} en el deque se demoró """
      f"""{deque_time:.6f} segundos.""")


# Vemos el time actual
start_time = time()
# Buscamos el elemento del medio
number_list[ELEMENTS // 2]
finish_time = time()
list_time = finish_time - start_time
# Imprimimos el tiempo transcurrido
print(f"""Buscar el elemento {ELEMENTS // 2} en la lista se demoró """
      f"""{list_time:.6f} segundos.""")
print(f"La búsqueda en deque fue {deque_time/list_time:.2f} veces el tiempo de list.")
print()

# Vamos a hacer pop de los primeros 1000 elementos del deque
start_time = time()
N = 1000
for i in range(0,N):
    number_deque.popleft()
finish_time = time()
deque_time = finish_time - start_time
print(f"Sacar los primeros {N} elementos del deque se demoró   {deque_time:.6f} segundos.")

# Vamos a hacer pop de los primeros 1000 elementos de la lista
start_time = time()
N = 1000
for i in range(0,N):
    number_list.pop(0)
finish_time = time()
list_time = finish_time - start_time
print(f"Sacar los primeros {N} elementos de la lista se demoró {list_time:.6f} segundos.")
print(f"La extracción en list fue {list_time/deque_time:.2f} veces el tiempo de deque.")
print()


Buscar el elemento 5000000 en el deque se demoró 0.004431 segundos.
Buscar el elemento 5000000 en la lista se demoró 0.000060 segundos.
La búsqueda en deque fue 74.35 veces el tiempo de list.

Sacar los primeros 1000 elementos del deque se demoró   0.000152 segundos.
Sacar los primeros 1000 elementos de la lista se demoró 7.044542 segundos.
La extracción en list fue 46457.47 veces el tiempo de deque.



Nuestro experimento nos permite apreciar que (1) **acceder a un elemento de la mitad** de una cola implementada con `list` es mucho más rápido que con `deque`, y (2) **extraer un elemento del inicio** de una cola implementada con `deque` es mucho más rápido que con `list`. Es importante conocer estas diferencias al momento de elegir una estructura de datos adecuada para nuestros programas.

#### Rotación

También existe una operación de rotación, que desplaza circularmente el _deque_. En la rotación, los `k` últimos elementos pasan a estar al principio como muestra este ejemplo:

In [9]:
from collections import deque

# Creamos un deque vacio y lo poblamos objeto a objeto.
d = deque()
d.append(0)
d.append(1)
d.append(2)
d.append(3)
d.append(4)
d.append(5)
d.append(6)

print(d)

# Rotamos el deque k=3. Los últimos k elementos pasan a ser los primeros
d.rotate(3)
print(d)

deque([0, 1, 2, 3, 4, 5, 6])
deque([4, 5, 6, 0, 1, 2, 3])


Se puede ver que con un _deque_ es posible simular un _stack_ o una cola como las que ya hemos visto. Esto pues un _deque_ puede hacer todas las operaciones que requieren ambas estructuras con la misma eficiencia. Es por eso que se dice que esta estructura es una **generalización de los _stacks_ y colas**.

### Ejemplo de uso con palíndromos

A continuación un ejemplo simple de chequeo de palabras palíndromas usando un _deque_. La palabra es almacenada en un _deque_ y las letras de los extremos son extraidas simultaneamente comparadas hasta que quede una sola letra.

In [7]:
from collections import deque

class Palabra:

    def __init__(self, palabra = None):
        self.letras = deque(palabra)

    def es_palindrome_rec(self):
        if len(self.letras) > 1:
            return self.letras.popleft() == self.letras.pop() \
        and Palabra(self.letras).es_palindrome_rec()
        else:
            return True

p1 = Palabra("reconocer")
p2 = Palabra("espectaculo")
p3 = Palabra("ana")
p4 = Palabra("OssO")

print(p1.es_palindrome_rec())
print(p2.es_palindrome_rec())
print(p3.es_palindrome_rec())
print(p4.es_palindrome_rec())


True
False
True
True


**Nota**: En Python lo más directo para chequear si un _string_ es palíndromo es simplemente comparar `palabra == palabra[::-1]`